In [78]:
import os
import datetime as dt
import glob

import colormaps
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

from contrails.units import pres2alt
from contrails.meteorology.era5 import get_ERA5_data
from contrails.satellites.goes.abi import get_nc_path, ABI2geodetic

DISTRIBUTION_DIR = "../data/flight_distribution_test/distributions/"

### Load flight altitude distributions

In [79]:
distribution_files = np.sort(glob.glob(os.path.join(DISTRIBUTION_DIR, "*.parquet")))

In [85]:
bins = np.arange(250, 450, 10)
img = np.full((len(bins), len(distribution_files)), 0.0)

counter = 0
for file in distribution_files:

    distribution = pd.read_parquet(file)
    
    # Normalize and multiply by 100 to get %
    distribution["dist_km"] /= distribution["dist_km"].sum()
    distribution["dist_km"] *= 100

    for k, v in distribution.iterrows():
        
        row = np.where(bins == int(k))[0]
        
        img[row, counter] = v.values[0]
        
    counter += 1

### Load test set results

In [172]:
res = pd.read_pickle("test_set_glad-resonance.pkl")

# These are the files in the test set with OpenSky coverage
test_set_files = pd.read_parquet("../data/test_files_glad-resonance.parquet")


res = res[res.filename.isin(test_set_files.file.values)]
res = res.groupby(res.filename).first().reset_index()

res['time'] = res.apply(lambda x: dt.datetime.strptime("_".join(
                    os.path.basename(x['filename']).split("_")[1:4]), "%Y%m%d_%H_%M"), axis=1)

# This is the filename of the flight distribution, if it exists
res['flight_fn'] = res['time'].dt.strftime("%Y%m%d_%H_%M.parquet")
res = res.reset_index()

# Keep only the rows with an existing flight distribution file
# and keep only the first pixel (as was done for generating the distributions, location-wise)
res = res[res.flight_fn.isin([os.path.basename(f) for f in distribution_files])]
res = res.groupby("flight_fn").first().reset_index()

order = res.y.argsort()
res = res.sort_values(by='y')

### Now convert the results to flight levels

First ensure that the quantiles are monotonically increasing

In [164]:
from sklearn.isotonic import IsotonicRegression

def make_monotonic(quantile_levels, quantile_values):
    """
    Based on the method in mcast/nowcasting/height_estimation/probabilistic.py
    
    Does not check for repeated values.
    TODO: Check for repeated values as this can give issues with the Pchip interpolator
    """
    iso_reg = IsotonicRegression().fit(quantile_levels, quantile_values)
    monotonic_quantile_values = iso_reg.predict(quantile_levels)

            
    return monotonic_quantile_values

quantile_columns = [c for c in res.columns if "q0" in c]
monotonic_quantile_columns = [col.replace("q","qm") for col in quantile_columns]
quantile_levels = np.array([float(col.lstrip("q")) for col in quantile_columns])

# Make sure to reset index: otherwise the addition of the columns will fail
res = res.reset_index()

res[monotonic_quantile_columns] = np.nan


res[monotonic_quantile_columns] = pd.DataFrame(res.apply(lambda x: make_monotonic(quantile_levels, x[quantile_columns].values),
                                                               axis=1).to_list(),
            columns=monotonic_quantile_columns)

In [168]:

def get_contrail_pressures(heights, gh_vals, pressures):
    contrail_pressures = np.zeros_like(heights)
    
    for i in range(len(heights)):
        
        contrail_pressures[i] = np.interp(heights[i], gh_vals, pressures)
        
        
    return contrail_pressures
    
def convert_row_to_flight_level(row):
    path = row.filename
    
    conus = False
    if "MCMIPC" in path:
        conus = True

    time = dt.datetime.strptime("_".join(os.path.basename(path).split("_")[1:4]), "%Y%m%d_%H_%M")
    min_col, max_col, min_row, max_row = [int(s) for s in os.path.basename(path).split(".")[0].split("_")[-4:]]
    label = np.load(path.replace("images", "labels"))

    rows, cols = np.where(label > 0)
    rows += min_row
    cols += min_col


    suffix = "C" if conus else "F"
    nc = xr.open_dataset(get_nc_path(time, product="ABI-L2-MCMIP" + suffix))

    x = nc.x.values[cols]
    y = nc.y.values[rows]

    lons, lats = ABI2geodetic(x, y)


    from collections import defaultdict

    itp_coords = {"longitude" : xr.DataArray(lons, dims="s"),
                  "latitude" : xr.DataArray(lats, dims="s"),
                  "time" : xr.DataArray(np.array([time] * len(lons)), dims="s")}
    
    
    era5 = get_ERA5_data(time)

    gh_vals = era5.z.interp(**itp_coords, method="linear").values[0,:]

    pressure_df = defaultdict(list)

    keys = monotonic_quantile_columns
    
    
    pressures = get_contrail_pressures(row[keys] * 1000, gh_vals / 9.80665, era5.isobaricInhPa.values)
    for k, v in zip(keys, pressures):

        pressure_df[k].append(v)
    
    pressure_df = pd.DataFrame(pressure_df)

    FL_df = pres2alt(pressure_df * 100) / 0.3048 / 100
    
    return FL_df.iloc[0]

In [169]:
FL_df = res.apply(convert_row_to_flight_level, axis=1)

Now create the boxes for the boxplots

In [170]:
boxes = []

for _, row in FL_df.iterrows():
    

    box = {"whislo" : row["qm0.025"],
           "q1" : 0.5 * (row["qm0.2"] + row["qm0.3"]),
           "med" : row["qm0.5"],
           "q3" : 0.5 * (row["qm0.7"] + row["qm0.8"]),
           "whishi" : row["qm0.975"],
          'fliers' : []}

    boxes.append(box)


### Create plot

In [ ]:
fig, ax = plt.subplots(dpi=500, figsize=(7.2, 7.2 * 0.5))


n = len(ordered_true_values)
vmin = 0
vmax = 30

cf = ax.imshow(img[::-1,order], extent=(0, img.shape[1], 245, 445), vmin=vmin, vmax=vmax, cmap=colormaps.cosmic)
counter = 0

# ordered_true_values[-1] -= 5

ls = ax.plot(np.arange(n) + 0.5, ordered_true_values, c='r',
       label="True value")


for box in boxes:
    
    bp = ax.bxp([box], positions=[counter+0.5], widths=0.5, boxprops=dict(color="w"),
                whiskerprops=dict(color="w"),
           medianprops=dict(color='w'), showcaps=False)
          
    counter += 1
    
    

ax.legend([bp["boxes"][0], ls[0]], ["CNN estimate", "True value"],
          facecolor="gray", framealpha=1.0)
    
    
ax.set_aspect("auto")

ax.set_xlim(0, img.shape[1])
ax.set_xticks(np.arange(0,len(boxes), 2)+0.5)
ax.set_xticklabels(np.arange(0,len(boxes), 2))
ax.set_ylim(250, 440)
ax.set_yticks(np.arange(250, 450, 20))
ax.set(xlabel="Data point index", ylabel="Flight level")

pos = ax.get_position()
cax = fig.add_axes([pos.x1+0.025, pos.y0, 0.025, pos.y1-pos.y0])

plt.colorbar(cf, cax=cax, label='% of distance flown in 2 hours before')